## Scoreing Fantasy Performance

1. read in gsheets
2. aggregate at the player - week level 
3. score by week
5. filter players with very low season long scores
4. write data 

In [1]:
#import library
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('../g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'

In [2]:
import pandas as pd
import numpy as np
from helper_funcs import passing_pts, receiving_rush_pts, writer

#### Passing

In [6]:
passing = client.open('passing').sheet1
passing = passing.get_all_values()
pass_df = pd.DataFrame(passing, columns = passing.pop(0))

In [7]:
floats = ['Comp', 'Att', 'Yds', 'TD', 'Int', 'Sck', 'FUM', 'Rate']

pass_df[floats] = pass_df[floats].astype(float)
pass_df["300yd_flag"] = np.where(pass_df['Yds'] >= 300, 1, 0)
pass_df['passing_fantasy_pts'] = passing_pts(pass_df)

In [8]:
pass_df.head()

,Name,Week,Team,Opp,Score,Comp,Att,Yds,TD,Int,Sck,FUM,Rate,300yd_flag,passing_fantasy_pts
0,Andy Dalton,1,CIN,@ SEA,L 20-21,35.0,51.0,418.0,2.0,0.0,5.0,2.0,106.5,1,25.72
1,Dak Prescott,1,DAL,vs NYG,W 35-17,25.0,32.0,405.0,4.0,0.0,0.0,0.0,158.3,1,35.20
2,Matthew Stafford,1,DET,@ ARI,T 27-27,27.0,45.0,385.0,3.0,0.0,3.0,1.0,110.0,1,29.40
3,Case Keenum,1,WAS,@ PHI,L 27-32,30.0,44.0,380.0,3.0,0.0,1.0,0.0,117.6,1,30.20
4,Patrick Mahomes,1,KC,@ JAX,W 40-26,25.0,33.0,378.0,3.0,0.0,0.0,0.0,143.2,1,30.12


#### Rushing 

In [9]:
rush = client.open('rushing').sheet1
rush = rush.get_all_values()
rush_df = pd.DataFrame(rush, columns = rush.pop(0))

In [10]:
rush_df[['Yds', 'TD', 'FUM']] = rush_df[['Yds', 'TD', 'FUM']].astype(float)
rush_df['100yd_flag'] = np.where(rush_df['Yds'] >= 100, 1, 0).astype(float)
rush_df['rush_fantasy_pts'] = receiving_rush_pts(rush_df)

In [11]:
rush_df.head()

,Name,Week,Team,Opp,Score,Att,Yds,Avg,TD,FUM,100yd_flag,rush_fantasy_pts
0,Marlon Mack,1,IND,@ LAC,L 24-30,25,174.0,7.0,1.0,0.0,1.0,23.4
1,Christian McCaffrey,1,CAR,vs LA,L 27-30,19,128.0,6.7,2.0,0.0,1.0,24.8
2,Saquon Barkley,1,NYG,@ DAL,L 17-35,11,120.0,10.9,0.0,0.0,1.0,12.0
3,Dalvin Cook,1,MIN,vs ATL,W 28-12,21,111.0,5.3,2.0,0.0,1.0,23.1
4,Mark Ingram,1,BAL,@ MIA,W 59-10,14,107.0,7.6,2.0,0.0,1.0,22.7


#### Receiving 

In [12]:
rec = client.open('receiving').sheet1
rec = rec.get_all_values()
rec_df = pd.DataFrame(rec, columns = rec.pop(0))

In [13]:
rec_df[['Yds', 'TD', 'FUM']] = rec_df[['Yds', 'TD', 'FUM']].astype(float)
rec_df['100yd_flag'] = np.where(rec_df['Yds'] >= 100, 1, 0).astype(float)
rec_df['rec_fantasy_pts'] = receiving_rush_pts(rec_df)

In [14]:
rec_df.head()

,Name,Week,Team,Opp,Score,Rec,Yds,Avg,TD,FUM,100yd_flag,rec_fantasy_pts
0,Sammy Watkins,1,KC,@ JAX,W 40-26,9,198.0,22.0,3.0,0.0,1.0,37.8
1,Michael Gallup,1,DAL,vs NYG,W 35-17,7,158.0,22.6,0.0,0.0,1.0,15.8
2,John Ross,1,CIN,@ SEA,L 20-21,7,158.0,22.6,2.0,0.0,1.0,27.8
3,DeSean Jackson,1,PHI,vs WAS,W 32-27,8,154.0,19.2,2.0,0.0,1.0,27.4
4,Marquise Brown,1,BAL,@ MIA,W 59-10,4,147.0,36.8,2.0,0.0,1.0,26.7


#### Filtering the data for low / no scores

In [15]:
pass_pts_agg = pass_df.groupby('Name')['passing_fantasy_pts'].sum().to_frame().rename(columns = {'passing_fantasy_pts':'szn_pts_total'})
pass_pts_agg.szn_pts_total = pass_pts_agg.szn_pts_total.astype(float)

In [18]:
print(pass_pts_agg.describe())

pass_filter = pass_pts_agg.loc[:, 'szn_pts_total'].quantile(.5)

print(f"points filter to remove players with very low season long totals: {pass_filter}")

       szn_pts_total
count     108.000000
mean       75.902222
std       105.433707
min        -1.000000
25%         0.430000
50%         6.940000
75%       184.650000
max       334.360000
points filter to remove players with very low season long totals: 6.9399999999999995


In [24]:
dropnames = pass_pts_agg.loc[pass_pts_agg['szn_pts_total'] < pass_filter]
pass_f = pass_df[~pass_df.Name.isin(dropnames)]

In [21]:
rec_agg = rec_df.groupby('Name')['rec_fantasy_pts'].sum().to_frame().rename(columns = {'rec_fantasy_pts':'szn_pts_total'})

rec_agg.szn_pts_total = rec_agg.szn_pts_total.astype(float)
dropnames = rec_agg.loc[rec_agg[ 'szn_pts_total'] < rec_agg.loc[:, 'szn_pts_total'].quantile(.5), :]
rec_f = rec_df[~rec_df.Name.isin(dropnames)]

In [22]:
rush_agg = rush_df.groupby('Name')['rush_fantasy_pts'].sum().to_frame().rename(columns = {'rush_fantasy_pts':'szn_pts_total'})

rush_agg.szn_pts_total = rush_agg.szn_pts_total.astype(float)
dropnames = rush_agg.loc[rush_agg[ 'szn_pts_total'] < rush_agg.loc[:, 'szn_pts_total'].quantile(.5), :]
rush_f = rush_df[~rush_df.Name.isin(dropnames)]

### Write the Data

In [28]:
writer(data = pass_f, sheet_name = 'passing_processed_step1', share_email = my_email)

writer(data = rush_f, sheet_name = 'rushing_processed_step1', share_email = my_email)

writer(data = rec_f, sheet_name = 'receiving_processed_step1', share_email = my_email)